In [ ]:
import pandas as pd
import ahocorasick

%load_ext autoreload

%autoreload 2

In [ ]:
df = pd.read_csv('/Users/kylegreen/codeup-data-science/trials_in_rust/trials-in-rust/first_rust_program/Testing/Almustaqbal.csv')
df.head()

In [ ]:
df.head(1).headline.values

In [ ]:
df.head(1).text

In [ ]:
df.head(1).text.values

In [ ]:
import pandas as pd
from googletrans import Translator
from camel_tools.sentiment import SentimentAnalyzer
from transformers import pipeline

In [ ]:
import pandas as pd
from googletrans import Translator
from camel_tools.sentiment import SentimentAnalyzer
from transformers import pipeline

#mix = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-mix-sentiment")
#ca = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-ca-sentiment")
#da = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment")
msa = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-sentiment")

#half = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-half")
#quarter = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-quarter")
#eighth = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-eighth")
#sixteenth = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-sixteenth")


In [ ]:

models = (mix, ca, da, msa, half, quarter, eighth, sixteenth)

In [ ]:
for mod in models:
    print(mod('hello'))

In [ ]:
df.head(5).text.values

In [ ]:
sum([len(val.split()) for val in df.head(100).text.values])/100
modelable = df.head(2000).text[[True if len(val.split()) < 350 else False for val in df.head(2000).text.values]]
for val in modelable.values:
    print(msa(val))


In [ ]:
for val in df.head(200).values:
    try:
        print(msa(val))
    except:
        continue
    
    #for mod in models:
        #print(mod(val[:512]))
        
subset = df.head(100).copy()

        
        
subset['label'] = subset.apply(lambda x:  msa(x.text)[0]['label'] if len(x.text.split()) < 300 else False, axis=1)
subset['score'] = subset.apply(lambda x: msa(x.text)[0]['score'] if len(x.text.split()) < 300 else False, axis=1)
subset.head()

#texts = {}    
#for i, val in enumerate(df.head(200).text.values):
#    if len(val) > 35 and len(val) < 512:
#        texts[i] = val
        
        
#print(texts)
        
        

In [ ]:
subset.head(15)

In [ ]:
subset.score.groupby(subset.label).max()

In [ ]:
subset[subset.label== 'negative'].max()

In [ ]:
translator = Translator()

In [ ]:
def translate_to_english(text):
    translation = translator.translate(text, dest='en').text
    return translation

translate_to_english(subset.head(1).text)

In [ ]:
subset.to_csv('labels.csv', index=False)

In [ ]:
sub = pd.read_csv('labels.csv')
sub.head()

In [ ]:
translator = Translator()
def translate_to_english(text):
    translation = translator.translate(text, dest='en').text
    return translation

def check_for_america(translation):
    if 'america' in translation.lower() or 'united states' in translation.lower():
        return True
    else:
        return False

text = sub.head(1).text



#translation = translate_to_english(text)
#check_for_america(translation)

row_nums = df.shape[0]
sets_of_ten = int(row_nums / 100)

collection_of_dfs = []
print(sets_of_ten)

for i in range(0,100):
    collection_of_dfs.append(df[df.index.isin(range(i * sets_of_ten, (i+1) * sets_of_ten))].copy())
    collection_of_dfs[i]['america'] = [True if val.str.contains('')]
    print(collection_of_dfs[f'frame_{i}'])
    break
    

#sub['en_text'] = sub.apply(lambda x: translate_to_english(x.text), axis=1)
#sub.head(10)

In [ ]:
df['source'] = 'almustaqbal'

In [ ]:
df[mask].to_csv('almustaqbal.csv')

In [ ]:
def flip_key_value_pairs(dicts):
    res = dict((v,k) for k,v in dicts.items())
    return res

def using_ahocorasick(col, lst):
    A = ahocorasick.Automaton(ahocorasick.STORE_INTS)
    for word in lst:
        A.add_word(word.lower())
    A.make_automaton() 
    col = col.astype(str)
    col = col.str.lower()
    mask = col.apply(lambda x: bool(list(A.iter(x))))
    tags = col.apply(lambda x: list(A.iter(x)))
    return mask, tags

def look_for_words_in_text(df_text):
    
    topics = {'America' : 'أمريكا',
            'American' : 'أمريكيّ',
            'American (f)' : 'أمريكيّة',
            'American (pl)' : 'أمريكيّين',
            'The United States' : 'الولايات المتحدة',
            'The United States' : 'دول موحّدة',
            'Washington' : 'واشنطن',
            'Bush' : 'بوش',
            'Obama' : 'أوباما',
            'Cheney' : 'تشيني',
            'Clinton' : 'كلينتون',
            'Osama Bin Laden' : 'أسامة بن لادن',
            'Al Gore' : 'آل غور',
            'World Trade Center' : 'مركز التجارة العالمي',
            '9/11' : '9/11',
            'September 11' : '11 سبتمبر',
            'Gulf War' : 'حرب الخليج',
            'Google' : 'غوغل',
            'Facebook' : 'فيسبوك',
            'Al Qaida' : 'القاعدة'}
    
    topics = flip_key_value_pairs(topics)
    
    tags = []
    for key in topics.keys():
        if key in df_text:
            tags.append(key)
            
    return tags

def make_relevant_tagged_df(df):
    topics = {'America' : 'أمريكا',
            'American' : 'أمريكيّ',
            'American (f)' : 'أمريكيّة',
            'American (pl)' : 'أمريكيّين',
            'The United States' : 'الولايات المتحدة',
            'The United States' : 'دول موحّدة',
            'Washington' : 'واشنطن',
            'Bush' : 'بوش',
            'Obama' : 'أوباما',
            'Cheney' : 'تشيني',
            'Clinton' : 'كلينتون',
            'Osama Bin Laden' : 'أسامة بن لادن',
            'Al Gore' : 'آل غور',
            'World Trade Center' : 'مركز التجارة العالمي',
            '9/11' : '9/11',
            'September 11' : '11 سبتمبر',
            'Gulf War' : 'حرب الخليج',
            'Google' : 'غوغل',
            'Facebook' : 'فيسبوك',
            'Al Qaida' : 'القاعدة'}


    topics = flip_key_value_pairs(topics)
    mask, tags = using_ahocorasick(df.text, list(topics.keys()))

    copied = df[mask].copy()

    copied['tags'] = copied.text.apply(look_for_words_in_text)

    return copied

publication_names = ['Alittihad', 'Almasryalyoum', 'Almustaqbal', 'Alqabas', 'Echoroukonline', 'Ryiadh', 'Sabanews', 'SaudiYoum', 'Techreen', 'Youm7']

filepath = '/Users/kylegreen/codeup-data-science/trials_in_rust/trials-in-rust/first_rust_program/Testing/'

for name in publication_names:
    path = filepath+name
    df = pd.read_csv(path + '.csv')
    df = make_relevant_tagged_df(df)
    df['source'] = name
    df.to_csv(name+'.csv', index=False)
    print(name)
print('done')

In [ ]:
for val in copied.tags.values:
    if len(val) > 1:
        print(val)

In [ ]:
sub['america'] = sub.apply(lambda x: check_for_america(x.en_text), axis=1)
sub.head(15)

In [ ]:
sub[sub.america==True].label.value_counts()

In [ ]:
pd.read_csv(publication_names[4] + '.csv')

In [ ]:
csvs = []
for name in publication_names:
    csvs.append(pd.read_csv(name + '.csv'))
df = pd.concat(csvs)
df.to_csv('all_articles.csv', index=False)

In [ ]:
df.source.value_counts()

In [ ]:
df.shape

In [ ]:
df.sample(500).to_csv('sample_articles.csv', index=False)

In [ ]:
steps = int(df.shape[0] /3) 

In [ ]:
split_dfs = []
for i in range(0, df.shape[0] - steps, steps):
    split_dfs.append(df.iloc[i: i+steps])
    
split_dfs[0].text.unique().size
    

In [ ]:
print(i)

In [ ]:
for i, d in enumerate(split_dfs):
    d.to_csv(f'block_{i+1}.csv', index=False)

In [ ]:
split_dfs[2].head()

In [ ]:
summarizer = pipeline('summarization')

In [ ]:
def make_msa(df_text):
    try:
        done = msa(df_text)
        return done
    except:
        try:
            first_half = msa(df_text[:round(len(df_text)/2)]) 
            second_half = msa(df_text[round(len(df_text)/2):])
            if first_half[0]['label'] == second_half[0]['label']:
                label = first_half[0]['label']
                score = (test_1[0]['score'] + test_2[0]['score'])/2
                done = [{'label': label, 'score': score}]
                return done
        except:
            print(df_text)
            return False
        

def analyze_text(df):
    scores = []
    scores = df.iloc[0:50].text.apply(make_msa)
    #df['score'] = df.text.apply(lambda x: msa(x)[0]['score'] if len(x.split()) < 300 else False)
    return scores

def analyze_headline(df):
    headline_scores = []
    scores = df.iloc[0:200].headline.apply(make_msa)
    return scores

#(analyze_text(df) != False).sum()
(analyze_text(subset_df) != False).sum()

In [ ]:
text= 'ائيل عن وثائق جديدة اليوم، السبت، عن أسرار حرب الاستنزاف التى خاضتها مصر ض'

In [ ]:
subset_df = pd.read_csv('sample_articles.csv')
subset_df.head()

In [ ]:
df.iloc[0]

In [ ]:
for val in subset_df.text.values:
    if text in val:
        print(subset_df[subset_df.text==val])

In [ ]:
source_dicts = {}
for source in df.source.unique():
    source_dicts[source] = df[df.source == source]
    
source_dicts['SaudiYoum']
    

In [ ]:
import re

In [ ]:
df.dateline.values[0]

In [ ]:
for val in list(df.dateline.values):
    try:
        print(re.search('20\d\d\W\d\d\W\d\d', val).group())
        
    except:
        print(val)

#print(len(df.dateline.values))

In [ ]:
source_dicts['SaudiYoum'].dateline.values[0]

In [ ]:


def using_ahocorasick_months(col, lst):
    A = ahocorasick.Automaton(ahocorasick.STORE_INTS)
    for word in lst:
        A.add_word(word.lower())
    A.make_automaton() 
    col = col.astype(str)
    col = col.str.lower()
    mask = col.apply(lambda x: bool(list(A.iter(x))))
    return mask

def look_for_words_in_text_months(df_text):
    
    months = {'January' : 'يناير',
                        'February' : 'فبراير',
                        'March' : 'مارس',
                        'April' : 'أبريل',
                        'June' : 'يوني',
                        'July' : 'يولي',
                        'August' : 'أغسطس',
                        'September' : 'سبتمبر',
                        'October' : 'أكتوبر',
                        'November' : 'نوفمبر',
                        'December' : 'ديسمبر',
                        'April1' : 'إبريل',
                        'May': 'ماي'}
    
    months = flip_key_value_pairs(months)
    
    tags = []
    for key in months.keys():
        if key in df_text:
            tags.append(key)
            
    return tags

mask = using_ahocorasick(df.dateline, list(months.values()))[0]
copy = df[mask].copy()
copy = pd.DataFrame(copy)
copy['month'] = copy.dateline.apply(look_for_words_in_text_months)


In [ ]:
copy

In [ ]:
def find_month(df_dateline):
    exp = '\d\d.*\d\d\d\d'
    try:
        month = re.search(exp, df_dateline).group()[2:-4]
        months = {'January' : 'يناير',
                        'February' : 'فبراير',
                        'March' : 'مارس',
                        'April' : 'أبريل',
                        'June' : 'يوني',
                        'July' : 'يولي',
                        'August' : 'أغسطس',
                        'September' : 'سبتمبر',
                        'October' : 'أكتوبر',
                        'November' : 'نوفمبر',
                        'December' : 'ديسمبر',
                        'April1' : 'إبريل',
                        'May': 'ماي'}
    
        months = flip_key_value_pairs(months)
        
        return re.search(exp, df_dateline).group()[2:-4]
    except:
        
        try:
            return re.search('20\d\d\W\d\d\W\d\d', val).group()

        except:
            return False

df.dateline.apply(find_month)

In [ ]:
df.dateline.size

In [ ]:
df=pd.read_csv('all_articles.csv')

In [1]:
import pandas as pd
from googletrans import Translator
from camel_tools.sentiment import SentimentAnalyzer
from transformers import pipeline


""" 
Change name to which file you're doing sentiment analysis on, NOT including the '.csv' 
Output file will be 'labeled_<filename>.csv'
"""

msa = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-sentiment")
path = 'split_articles/'

def load_and_label_df(name):
    df = load_csv(path + name)
    print(f'loaded {name}')
    print('labeling/scoring...')
    df = create_labels_scores(df, name)
    print('done labeling/scoring!')
    return df

def load_csv(filename):
    df = pd.read_csv(filename)
    return df

def make_msa(df_text):
    try:
        done = msa(df_text)
        return done
    except:
        
        try:
            first_half = msa(df_text[:round(len(df_text)/2)]) 
            second_half = msa(df_text[round(len(df_text)/2):])
            if first_half[0]['label'] == second_half[0]['label']:
                label = first_half[0]['label']
                score = (test_1[0]['score'] + test_2[0]['score'])/2
            done = [{'label': label, 'score': score}]
            return done
        except:
        
            try:
                beginning = msa(df_text[:round(len(df_text)/3)]) 
                middle = msa(df_text[round(len(df_text)/3):round(len(df_text)*2/3)])
                end = msa(df_text[round(len(df_text)*2/3):])

                if (beginning[0]['label'] == middle[0]['label']) and (beginning[0]['label'] == end[0]['label']):
                    label = first_half[0]['label']
                    score = (beginning[0]['score'] + middle[0]['score'] + end[0]['score'])/3
                    done = [{'label': label, 'score': score}]
                    return done
                return False
            except:
                return False
        
def analyze_text(df):
    scores = []
    print('analyzing_texts')
    scores = [make_msa(val) for val in df.text.values]
    print(scores)
    return scores

def analyze_headline(df):
    print('analyzing headlines')
    headline_scores = [make_msa(val) for val in df.headline.values]
    return headline_scores

def label_and_scores(msa_scores):
    labels = []
    scores = []
    for val in msa_scores:
        print(val)
        try:
            labels.append(val[0]['label'])
            scores.append(val[0]['score'])
        except:
            labels.append(False)
            scores.append(False)

    return labels, scores

def create_labels_scores(df, name):
    text_scores = analyze_text(df)
    labels, scores = label_and_scores(text_scores)
    df['text_label'] = labels
    df['text_score'] = scores

    headline_scores = analyze_headline(df)
    labels, scores = label_and_scores(headline_scores)
    df['headline_label'] = labels
    df['headline_score'] = scores

    # CHANGE 'BLOCK_NAME' TO WHATEVER YOU WANT
    df.to_csv('labeled_split_articles/labeled_'+ name + '.csv', index=False)

    return df

#load_and_label_df(name)




In [ ]:
pd.read_csv('labeled_' +name+'.csv')

In [ ]:
def split_tons_of_csvs():
    df = pd.read_csv('/Users/kylegreen/codeup-data-science/all_articles.csv')
    
    end_point = '/Users/kylegreen/codeup-data-science/arabic_media_nlp_project/kyles_work/split_articles/'
    
    df_shape = df.shape[0]
    
    one_thousandth = int(df_shape/1000)
    
    for i in range(0, df_shape, one_thousandth):
        copy = df.iloc[i:i+one_thousandth].copy()
        copy.to_csv(end_point+str(i) + '.csv', index=False)
    
split_tons_of_csvs().head()

In [ ]:
pd.read_csv('split_articles/354501.csv')

In [ ]:
import os
import regex as re

labeled = 'labeled_split_articles'
unlabeled = 'split_articles'

os.listdir(labeled)

unlabeled_files = os.listdir(unlabeled)
labeled_files = []

for path in os.listdir(labeled):
    labeled_files.append(re.search('\d*\.', path).group()[:-1])

    
ul_files = unlabeled_files[:]

for p in labeled_files:
    for pth in unlabeled_files:
        if p in pth:
            ul_files.remove(pth)
            
path = 'split_articles/'
for file in ul_files:
    load_and_label_df(file)

354501.csv
loaded 255255.csv
labeling/scoring...
analyzing_texts
[[{'label': 'neutral', 'score': 0.8719582557678223}], [{'label': 'negative', 'score': 0.7396730979283651}], [{'label': 'neutral', 'score': 0.8675128221511841}], [{'label': 'negative', 'score': 0.9959478974342346}], False, [{'label': 'neutral', 'score': 0.9681220054626465}], False, False, False, False, False, [{'label': 'neutral', 'score': 0.9735515713691711}], False, [{'label': 'neutral', 'score': 0.8790179093678793}], [{'label': 'negative', 'score': 0.9960763653119405}], [{'label': 'negative', 'score': 0.8343658645947775}], [{'label': 'negative', 'score': 0.9952484965324402}], [{'label': 'negative', 'score': 0.7642999092737833}], False, False, [{'label': 'negative', 'score': 0.8894286950429281}], [{'label': 'negative', 'score': 0.7731797695159912}], [{'label': 'neutral', 'score': 0.7473170161247253}], False, [{'label': 'neutral', 'score': 0.9807820320129395}], False, False, [{'label': 'neutral', 'score': 0.95078921318054

[{'label': 'negative', 'score': 0.8795162439346313}]
[{'label': 'neutral', 'score': 0.6621490120887756}]
[{'label': 'neutral', 'score': 0.993306577205658}]
[{'label': 'negative', 'score': 0.994967520236969}]
[{'label': 'neutral', 'score': 0.7059893012046814}]
[{'label': 'neutral', 'score': 0.9798362255096436}]
[{'label': 'negative', 'score': 0.949890673160553}]
[{'label': 'negative', 'score': 0.6844911575317383}]
[{'label': 'neutral', 'score': 0.9956859350204468}]
[{'label': 'neutral', 'score': 0.9948793649673462}]
[{'label': 'neutral', 'score': 0.8663921356201172}]
[{'label': 'neutral', 'score': 0.9694013595581055}]
[{'label': 'negative', 'score': 0.5795223712921143}]
[{'label': 'neutral', 'score': 0.964597761631012}]
[{'label': 'negative', 'score': 0.5350624322891235}]
[{'label': 'negative', 'score': 0.9518306255340576}]
[{'label': 'neutral', 'score': 0.9700462818145752}]
[{'label': 'neutral', 'score': 0.943327009677887}]
[{'label': 'neutral', 'score': 0.9752681255340576}]
[{'label':

[[{'label': 'neutral', 'score': 0.7481210231781006}], [{'label': 'neutral', 'score': 0.9499773979187012}], [{'label': 'neutral', 'score': 0.7549616098403931}], [{'label': 'positive', 'score': 0.938532292842865}], [{'label': 'neutral', 'score': 0.8847028017044067}], [{'label': 'neutral', 'score': 0.919029712677002}], False, [{'label': 'negative', 'score': 0.6146898865699768}], [{'label': 'negative', 'score': 0.987639844417572}], [{'label': 'negative', 'score': 0.8915374676386515}], [{'label': 'negative', 'score': 0.7064467668533325}], [{'label': 'neutral', 'score': 0.7773579359054565}], False, False, [{'label': 'neutral', 'score': 0.9768158992131551}], [{'label': 'neutral', 'score': 0.9475659330685934}], [{'label': 'neutral', 'score': 0.8307480216026306}], [{'label': 'neutral', 'score': 0.796640932559967}], [{'label': 'neutral', 'score': 0.7106949687004089}], [{'label': 'negative', 'score': 0.8304899533589681}], [{'label': 'negative', 'score': 0.5975235104560852}], [{'label': 'neutral',

[{'label': 'negative', 'score': 0.9829708337783813}]
[{'label': 'neutral', 'score': 0.9172180891036987}]
[{'label': 'neutral', 'score': 0.9448040127754211}]
[{'label': 'positive', 'score': 0.7847399711608887}]
[{'label': 'neutral', 'score': 0.8510077595710754}]
[{'label': 'neutral', 'score': 0.932781457901001}]
[{'label': 'negative', 'score': 0.7231534719467163}]
[{'label': 'neutral', 'score': 0.6031057834625244}]
[{'label': 'negative', 'score': 0.8368291258811951}]
[{'label': 'negative', 'score': 0.8955113887786865}]
[{'label': 'negative', 'score': 0.8805517554283142}]
[{'label': 'neutral', 'score': 0.8716725707054138}]
[{'label': 'negative', 'score': 0.7451223134994507}]
[{'label': 'neutral', 'score': 0.9322944283485413}]
[{'label': 'negative', 'score': 0.7685848474502563}]
[{'label': 'neutral', 'score': 0.7512106895446777}]
[{'label': 'neutral', 'score': 0.9889863729476929}]
[{'label': 'neutral', 'score': 0.6120957136154175}]
[{'label': 'positive', 'score': 0.5625711679458618}]
[{'l

[[{'label': 'neutral', 'score': 0.7860100865364075}], [{'label': 'neutral', 'score': 0.8218486905097961}], [{'label': 'neutral', 'score': 0.8830838799476624}], False, [{'label': 'neutral', 'score': 0.9848211407661438}], False, [{'label': 'neutral', 'score': 0.9742153882980347}], [{'label': 'negative', 'score': 0.8226291537284851}], False, [{'label': 'positive', 'score': 0.8639079928398132}], False, [{'label': 'neutral', 'score': 0.9193724393844604}], [{'label': 'neutral', 'score': 0.7326289415359497}], [{'label': 'neutral', 'score': 0.800940752029419}], [{'label': 'neutral', 'score': 0.8968347311019897}], False, [{'label': 'neutral', 'score': 0.9901556968688965}], [{'label': 'neutral', 'score': 0.9283944169680277}], [{'label': 'neutral', 'score': 0.9713710347811381}], [{'label': 'neutral', 'score': 0.984987199306488}], [{'label': 'neutral', 'score': 0.8697119355201721}], [{'label': 'neutral', 'score': 0.9715533256530762}], [{'label': 'neutral', 'score': 0.9011834263801575}], [{'label':

...

[{'label': 'neutral', 'score': 0.9715369939804077}]
[{'label': 'neutral', 'score': 0.9871872663497925}]
[{'label': 'neutral', 'score': 0.56460040807724}]
[{'label': 'neutral', 'score': 0.9632691144943237}]
[{'label': 'negative', 'score': 0.9885614514350891}]
[{'label': 'neutral', 'score': 0.6641183495521545}]
[{'label': 'neutral', 'score': 0.9095481634140015}]
[{'label': 'neutral', 'score': 0.7197248935699463}]
[{'label': 'neutral', 'score': 0.7197248935699463}]
[{'label': 'neutral', 'score': 0.9911988377571106}]
[{'label': 'neutral', 'score': 0.8884870409965515}]
[{'label': 'neutral', 'score': 0.9726417660713196}]
[{'label': 'neutral', 'score': 0.8744903206825256}]
[{'label': 'neutral', 'score': 0.9345563650131226}]
[{'label': 'neutral', 'score': 0.8525623083114624}]
[{'label': 'neutral', 'score': 0.7481322288513184}]
[{'label': 'positive', 'score': 0.5534255504608154}]
[{'label': 'neutral', 'score': 0.9848020672798157}]
[{'label': 'neutral', 'score': 0.9919214248657227}]
[{'label': '

[[{'label': 'negative', 'score': 0.8379503488540649}], False, [{'label': 'negative', 'score': 0.9015089273452759}], [{'label': 'negative', 'score': 0.9419634342193604}], [{'label': 'neutral', 'score': 0.5402606129646301}], False, False, False, [{'label': 'neutral', 'score': 0.9758206605911255}], [{'label': 'neutral', 'score': 0.9677280187606812}], [{'label': 'neutral', 'score': 0.9075939059257507}], [{'label': 'neutral', 'score': 0.9203643798828125}], [{'label': 'neutral', 'score': 0.9724076787630717}], [{'label': 'neutral', 'score': 0.87484210729599}], False, False, False, [{'label': 'neutral', 'score': 0.9858321150143942}], [{'label': 'neutral', 'score': 0.9575199484825134}], False, False, [{'label': 'negative', 'score': 0.9066935976346334}], False, False, [{'label': 'neutral', 'score': 0.965621292591095}], False, [{'label': 'neutral', 'score': 0.5848086476325989}], [{'label': 'neutral', 'score': 0.9174344539642334}], [{'label': 'neutral', 'score': 0.9745495915412903}], [{'label': 'n

[{'label': 'neutral', 'score': 0.6868988275527954}]
[{'label': 'neutral', 'score': 0.9834097623825073}]
[{'label': 'negative', 'score': 0.9769282937049866}]
[{'label': 'negative', 'score': 0.9594640731811523}]
[{'label': 'neutral', 'score': 0.7790851593017578}]
[{'label': 'neutral', 'score': 0.9632734656333923}]
[{'label': 'neutral', 'score': 0.8666865229606628}]
[{'label': 'negative', 'score': 0.6795392632484436}]
[{'label': 'neutral', 'score': 0.9937044978141785}]
[{'label': 'neutral', 'score': 0.9014727473258972}]
[{'label': 'negative', 'score': 0.6569004058837891}]
[{'label': 'neutral', 'score': 0.9612289071083069}]
[{'label': 'neutral', 'score': 0.9839416146278381}]
[{'label': 'negative', 'score': 0.9221221804618835}]
[{'label': 'neutral', 'score': 0.989510178565979}]
[{'label': 'neutral', 'score': 0.7524261474609375}]
[{'label': 'neutral', 'score': 0.9720028042793274}]
[{'label': 'negative', 'score': 0.9729462265968323}]
[{'label': 'positive', 'score': 0.5228468775749207}]
[{'lab

[False, False, False, False, [{'label': 'negative', 'score': 0.9906991124153137}], [{'label': 'neutral', 'score': 0.994371771812439}], [{'label': 'neutral', 'score': 0.6474819779396057}], [{'label': 'neutral', 'score': 0.7242356538772583}], [{'label': 'neutral', 'score': 0.49278539419174194}], False, False, False, [{'label': 'negative', 'score': 0.8397748668988546}], [{'label': 'negative', 'score': 0.8797754049301147}], [{'label': 'neutral', 'score': 0.978118896484375}], [{'label': 'neutral', 'score': 0.8658257126808167}], [{'label': 'neutral', 'score': 0.9866650104522705}], [{'label': 'positive', 'score': 0.7361931800842285}], [{'label': 'negative', 'score': 0.9394885897636414}], [{'label': 'neutral', 'score': 0.9893577694892883}], [{'label': 'neutral', 'score': 0.9717725515365601}], [{'label': 'neutral', 'score': 0.8680812120437622}], [{'label': 'negative', 'score': 0.8684436082839966}], [{'label': 'neutral', 'score': 0.8558836579322815}], [{'label': 'neutral', 'score': 0.90961223840

[{'label': 'neutral', 'score': 0.8227882981300354}]
[{'label': 'neutral', 'score': 0.9597955346107483}]
[{'label': 'negative', 'score': 0.7154032588005066}]
[{'label': 'positive', 'score': 0.9783888459205627}]
[{'label': 'neutral', 'score': 0.9391583204269409}]
[{'label': 'neutral', 'score': 0.9945300221443176}]
[{'label': 'negative', 'score': 0.9351090788841248}]
[{'label': 'negative', 'score': 0.9453582167625427}]
[{'label': 'positive', 'score': 0.5535736083984375}]
[{'label': 'neutral', 'score': 0.9842144250869751}]
[{'label': 'negative', 'score': 0.8884705901145935}]
[{'label': 'negative', 'score': 0.6645421385765076}]
[{'label': 'negative', 'score': 0.9946146607398987}]
[{'label': 'neutral', 'score': 0.9230895638465881}]
[{'label': 'neutral', 'score': 0.7605047821998596}]
[{'label': 'negative', 'score': 0.7655165195465088}]
[{'label': 'neutral', 'score': 0.9809156060218811}]
[{'label': 'neutral', 'score': 0.988684356212616}]
[{'label': 'negative', 'score': 0.9059469699859619}]
[{'

In [ ]:
for fi in os.listdir('labled_split_articles'):
    print()

In [ ]:
import regex as re
import pandas as pd

def find_month(df_dateline):
    exp = '\d\d.*\d\d\d\d'
    try:
        month = re.search(exp, df_dateline).group()[2:-4]
        months = {'January' : 'يناير',
                        'February' : 'فبراير',
                        'March' : 'مارس',
                        'April' : 'أبريل',
                        'June' : 'يوني',
                        'July' : 'يولي',
                        'August' : 'أغسطس',
                        'September' : 'سبتمبر',
                        'October' : 'أكتوبر',
                        'November' : 'نوفمبر',
                        'December' : 'ديسمبر',
                        'April1' : 'إبريل',
                        'May': 'ماي'}
    
        months = flip_key_value_pairs(months)
        
        return re.search(exp, df_dateline).group()[2:-4]
    except:
        
        try:
            return re.search('20\d\d\W\d\d\W\d\d', val).group()

        except:
            return False


def flip_key_value_pairs(dicts):
    res = dict((v,k) for k,v in dicts.items())
    return res

df.head()